In [1]:
from library.common import Core, Database
import pandas as pd

def clean_column_names(df):
    _new_col = []
    for _col in df.columns:
        _col = _col.lower()
        _col = _col.replace(' ', '_')
        _col = _col.replace(r'%', "_pct")
        _new_col.append(_col)
    return _new_col

db = Database()
core = Core()

db_table = 'consolidated_data_csv'
csv = pd.read_sql(db_table, db.config)
csv.columns = clean_column_names(csv)
csv = csv.rename({'year': 'csv_year'}, axis = 1)
csv_list = list(csv.country_code.unique())

db_table = 'owid_co2_greenhouse_gas_emissions'
owid = pd.read_sql(db_table, db.engine)
owid_list = list(owid.iso_code.unique())
owid.columns = clean_column_names(owid)

print('Data loaded from DATABASE')

Data loaded from DATABASE


In [ ]:
final_list = set(csv_list).intersection(set(owid_list))

o_df = owid[owid.iso_code.isin(final_list)].copy()
print(owid.shape, o_df.shape)
c_df = csv[csv.country_code.isin(final_list)].copy()
print(csv.shape, c_df.shape)


c_key = ['country_code', 'csv_year']
o_key = ['iso_code', 'year']

final_df = o_df.merge(c_df, how = 'left',  left_on= o_key, right_on= c_key, validate = '1:1').fillna(0)
final_df = final_df.drop(labels = ['country_code', 'csv_year', 'country_name'], axis = 1)

print("Data manipulation COMPLETED")

table = 'master_data_source'
final_df.to_sql(table, db.config, index = False, if_exists = 'replace')
print("MASTER DATA loaded to database")

(25204, 58) (20798, 58)
(13298, 51) (12383, 51)
Data manipulation COMPLETED
